# **Cartes extraites du Malaria Atlas Project (MAP)**

In [ ]:
# Set SNT Paths
SNT_ROOT_PATH  <- "~/workspace"
CODE_PATH      <- file.path(SNT_ROOT_PATH, "code")
CONFIG_PATH    <- file.path(SNT_ROOT_PATH, "configuration")

# load util functions
source(file.path(CODE_PATH, "snt_utils.r"))

# List required packages 
required_packages <- c("dplyr", "tidyr", "terra", "ggplot2", "stringr", "lubridate", "viridis", "patchwork", "zoo", "purrr", "arrow", "sf", "reticulate")

# Execute function
install_and_load(required_packages)

# Set environment to load openhexa.sdk from the right environment
Sys.setenv(RETICULATE_PYTHON = "/opt/conda/bin/python")
reticulate::py_config()$python
openhexa <- import("openhexa.sdk")

# Load SNT config
config_json <- tryCatch({ jsonlite::fromJSON(file.path(CONFIG_PATH, "SNT_config.json"))},
    error = function(e) {
        msg <- paste0("Error while loading configuration", conditionMessage(e))  
        cat(msg)   
        stop(msg) 
    })

# Required environment for the sf packages
Sys.setenv(PROJ_LIB = "/opt/conda/share/proj")
Sys.setenv(GDAL_DATA = "/opt/conda/share/gdal")

In [ ]:
# Configuration variables
DATASET_NAME <- config_json$SNT_DATASET_IDENTIFIERS$SNT_MAP_EXTRACT
COUNTRY_CODE <- config_json$SNT_CONFIG$COUNTRY_CODE
ADM_2 <- toupper(config_json$SNT_CONFIG$DHIS2_ADMINISTRATION_2)

In [ ]:
# print function
printdim <- function(df, name = deparse(substitute(df))) {
  cat("Dimensions of", name, ":", nrow(df), "rows x", ncol(df), "columns\n\n")
}

In [ ]:
# import seasonality data
map_data <- tryCatch({ get_latest_dataset_file_in_memory(DATASET_NAME, paste0(COUNTRY_CODE, "_map_data.parquet")) }, 
                  error = function(e) {
                      msg <- paste("Error while loading seasonality file for: " , COUNTRY_CODE, conditionMessage(e))
                      cat(msg)
                      stop(msg)
                      })
# import DHIS2 shapes data
DATASET_DHIS2 <- config_json$SNT_DATASET_IDENTIFIERS$DHIS2_DATASET_FORMATTED
shapes_data <- tryCatch({ get_latest_dataset_file_in_memory(DATASET_DHIS2, paste0(COUNTRY_CODE, "_shapes.geojson")) }, 
                  error = function(e) {
                      msg <- paste("Error while loading DHIS2 Shapes data for: " , COUNTRY_CODE, conditionMessage(e))
                      cat(msg)
                      stop(msg)
                      })

printdim(map_data)

In [ ]:
names(map_data)

In [ ]:
unique(map_data$METRIC_NAME)

In [ ]:
# Merge geometry with map data
map_data_joined <- left_join(shapes_data, map_data, by = c("ADM2_ID" = "ADM2_ID"))

# Get list of metrics
metrics <- unique(map_data$METRIC_NAME)

# Create one map per metric
plots <- map(metrics, function(metric) {
  ggplot(map_data_joined %>% filter(METRIC_NAME == metric)) +
    geom_sf(aes(fill = VALUE), color = "white") +
    scale_fill_viridis_c(option = "C", na.value = "lightgrey") +
    labs(
      title = paste0(metric),
      fill = "Valeur"
    ) +
    theme_minimal(base_size = 16) +
    theme(
      plot.title = element_text(size = 20, face = "bold"),
      legend.title = element_text(size = 16),
      legend.text = element_text(size = 14)
    )
})

In [ ]:
# Set plot size for individual display
options(repr.plot.width = 10, repr.plot.height = 8)

# Loop through plots and print one by one
for (p in plots) {
  print(p)
  Sys.sleep(1)  # Optional: short pause between plots
}